In [76]:
import codecs,unidecode
import itertools as it

In [122]:
fEtudiants="/Users/gilles/Desktop/L1-Quizz.csv"
fQuizzoodle="/Users/gilles/Desktop/Quizz8.csv"

In [78]:
alias={}

In [79]:
def assemblerNoms(nom):
    noms=nom.replace(" ",";").replace("-",";").split(";")
    result=[]
    for ordre in it.permutations(noms):
        result.append("".join(ordre))
    return result

In [80]:
assemblerNoms("Gilles;Le Boyé")

['GillesLeBoy\xc3\xa9',
 'GillesBoy\xc3\xa9Le',
 'LeGillesBoy\xc3\xa9',
 'LeBoy\xc3\xa9Gilles',
 'Boy\xc3\xa9GillesLe',
 'Boy\xc3\xa9LeGilles']

In [81]:
with codecs.open(fEtudiants,"r",encoding="utf8") as input:
    lEtudiants=input.readlines()
etudiants={}
for ligne in lEtudiants:
    nom=ligne.strip()
    for etudiant in assemblerNoms(nom):
        etudiants[etudiant]=nom
etudiants

{u'ABDULLATUFHONORINE': u'ABDULLATUF;HONORINE',
 u'ABESJAUTZYPAUL': u'ABES-JAUTZY;PAUL',
 u'ABESPAULJAUTZY': u'ABES-JAUTZY;PAUL',
 u'ABOVICICHARLEEN': u'ABOVICI;CHARLEEN',
 u'ACHILLEBOUVIN': u'BOUVIN;ACHILLE',
 u'ADELEBARDIN': u'BARDIN;ADELE',
 u'ADRIENDELAYE': u'DELAYE;ADRIEN',
 u'AGATHEBASSE': u'BASSE;AGATHE',
 u'AKILABENKAMARDINE': u'BEN KAMARDINE;AKILA',
 u'AKILAKAMARDINEBEN': u'BEN KAMARDINE;AKILA',
 u'ALEXANDREAUTHIER': u'AUTHIER;ALEXANDRE',
 u'ALEXIABELIVIER': u'BELIVIER;ALEXIA',
 u'ALEXISDUNIAU': u'DUNIAU;ALEXIS',
 u'ALICEBAZALGETTEMOIROT': u'BAZALGETTE-MOIROT;ALICE',
 u'ALICEMOIROTBAZALGETTE': u'BAZALGETTE-MOIROT;ALICE',
 u'ALICIALINARES': u'LINARES;ALICIA',
 u'ALICIAVOISIN': u'VOISIN;ALICIA',
 u'ALYZEEFARA': u'FARA;ALYZEE',
 u'AMANDINEBERDOT': u'BERDOT;AMANDINE',
 u'AMELIEARNAUD': u'ARNAUD;AMELIE',
 u'AMPUYDAROLLESJULIA': u'DAROLLES-AMPUY;JULIA',
 u'AMPUYJULIADAROLLES': u'DAROLLES-AMPUY;JULIA',
 u'ANDRALMARIE': u'ANDRAL;MARIE',
 u'ANNADELVERT': u'DELVERT;ANNA',
 u'ANNEDUPELSO

In [105]:
def chercheAlias(nom):
    alias=[]
    chaine=unidecode.unidecode(nom).replace(" ","").replace("-","").upper()
    if chaine in etudiants:
        alias=etudiants[chaine]
    else:
        while not alias:
            saisie=raw_input("quel nom pour %s"%chaine)
            saisie = unidecode.unidecode(saisie).upper()
            if saisie=="?":
                alias="?"
            elif " " in saisie:
                alias=saisie.split(" ")
                if not all(a in etudiants for a in alias):
                    alias=[]
                else:
                    alias=[etudiants[a] for a in alias]
                    print alias
            elif saisie in etudiants:
                alias=etudiants[saisie]
    return alias

In [94]:
chercheAlias(u"adèle bardin")

u'BARDIN;ADELE'

In [123]:
with codecs.open(fQuizzoodle,"r",encoding="utf8") as input:
    lQuizz=input.readlines()
notes={}
for ligne in lQuizz[1:]:
#    print ligne
    nom,bons,temps=ligne.strip().split(";")
    if not nom in alias:
        aka=chercheAlias(nom)
        if isinstance(aka,list):
            for a in aka:
                alias[nom]=a
                notes[a]=round(float(bons)/6*10+10)
        else:
            alias[nom]=aka
            notes[aka]=round(float(bons)/6*10+10)
    else:
        notes[alias[nom]]=round(float(bons)/6*10+10)
            

quel nom pour ENORABLEGERENORA


/Users/gilles/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:9: RuntimeWarning: Argument <type 'str'> is not an unicode object. Passing an encoded string will likely have unexpected results.
  if __name__ == '__main__':


quel nom pour DERAILMARINEDERAIL
quel nom pour ROXANE.VIGOUROUXROXANEVIGOUROUX
quel nom pour LEMORPHOLOGUEDESERVICE?
quel nom pour ELSAGUILLOUELSALE
quel nom pour GREGETPAUL?


In [124]:
notes

{'?': 12.0,
 u'AUDOUEINEIX;CAMILLE': 12.0,
 u'BASSE;JADE': 13.0,
 u'BELHANI;MARINE': 13.0,
 u'BELIVIER;ALEXIA': 15.0,
 u'BERNARD-ROUYER;ELINA': 10.0,
 u'BEZEMONT-JUZAN;NATHAN': 13.0,
 u'BLEGER;ENORA': 15.0,
 u'BOURGMEYER;MANON': 15.0,
 u'CARBONNIER;EVANE': 15.0,
 u'CARQUIN;CLEMENCE': 15.0,
 u'CHALLANCIN;CARLA': 13.0,
 u'CHARLES;EMILIE': 15.0,
 u'CODOGNOTTO;EMILIE': 12.0,
 u'DERAIL;MARINE': 10.0,
 u'DIALLO;MAMADOU OPA': 10.0,
 u'DUMAINE;CHARLES': 13.0,
 u'DUPEL;ANNE-SOPHIE': 15.0,
 u'ELOSEGUI-MARTINON;UXUE': 13.0,
 u'GENDRE-DESCRAMBES;DYLAN': 15.0,
 u'GUILLON;SHANEZE': 15.0,
 u'HIM;LORENE': 13.0,
 u'JACOB-HIPPLER;JADE': 13.0,
 u'JACQUEMIN;CLARISSE': 12.0,
 u'JANOULET;PAULINE': 10.0,
 u'JAUBERT;MARIE': 12.0,
 u'LAMANE;TIFFANY': 10.0,
 u'LANDURE;ELISE': 13.0,
 u'LATERRADE;LENA': 13.0,
 u'LE GUILLOU;ELSA': 12.0,
 u'LE LAURAIN;TOM': 13.0,
 u'MAGNAC;OCEANE': 12.0,
 u'MAILLARD;CELIA': 15.0,
 u'MICHELET;LUCIE': 13.0,
 u'MONFROY;MARINA': 12.0,
 u'MONNIER;MAILYS': 13.0,
 u'NOAILLES;MORGANE': 15.

In [92]:
alias

{u':) ': '?',
 u'Abovici charleen ': u'ABOVICI;CHARLEEN',
 u'Ad\xe8le Bardin ': u'BARDIN;ADELE',
 u'Amndine Berdot ': u'BERDOT;AMANDINE',
 u'Anne-Sophie dupel ': u'DUPEL;ANNE-SOPHIE',
 u'Audoueineix ': u'AUDOUEINEIX;CAMILLE',
 u'Basse Jade ': u'BASSE;JADE',
 u'Bourgmeyer manon ': u'BOURGMEYER;MANON',
 u'Carquin cl\xe9mence ': u'CARQUIN;CLEMENCE',
 u'Charles Dumaine ': u'DUMAINE;CHARLES',
 u'ELINA BERNARD ROUYER ': u'BERNARD-ROUYER;ELINA',
 u'EddyHernandezL1SDL ': u'HERNANDEZ;EDDY',
 u'ElisaLAGRANGE ': u'LAGRANGE;ELISA',
 u'Elise LANDURE ': u'LANDURE;ELISE',
 u'Elosegui ': u'ELOSEGUI-MARTINON;UXUE',
 u'Gendre-Descrambes Dylan ': u'GENDRE-DESCRAMBES;DYLAN',
 u'HIM Lor\xe8ne ': u'HIM;LORENE',
 u'JDangoumau ': u'DANGOUMAU;JUDITH',
 u'Jacob-Hippler ': u'JACOB-HIPPLER;JADE',
 u'Jacquemin Pochelu susset ': u'SUSSET;MANON',
 u'Justine T ': u'TENET;JUSTINE',
 u'Lena Laterrade ': u'LATERRADE;LENA',
 u'Livya ': u'RAMAYE;LIVIA',
 u'Maeva Triffaux ': u'TRIFFAUX;MAEVA',
 u'Maillard ': u'MAILLARD;CEL

In [97]:
del alias[u'Jacquemin Pochelu susset ']

In [98]:
del alias[u'\xc9milie CHARLES Am\xe9lie ARNAUD ']